In [1]:
import numpy as np
import torch as th
from concurrent.futures import ProcessPoolExecutor

from glob import glob
from utils import *
import ray
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


df_result = pd.read_csv("./result.csv", index_col=0)
sigs_name = df_result.index.tolist()
sigs_dir = [
    sorted(
        glob(f"checkpoints/{name}/*.json"),
        key=lambda x: int(x.split("/")[-1].split("_")[0]),
    )[-1]
    for name in sigs_name
]


sigs_dict = {}
with ProcessPoolExecutor(80) as executor:
    for name, value in zip(
        sigs_name,
        executor.map(
            json_to_factor,
            sigs_dir,
            [20190103] * len(sigs_name),
            [20211231] * len(sigs_name),
        ),
        strict=True,
    ):
        sigs_dict[name] = value


In [2]:
sigs = th.stack(list(sigs_dict.values()),dim=2)

In [3]:
sigs.shape

torch.Size([11548, 6000, 101])

In [4]:
sigs = th.concat([th.zeros(100,sigs.shape[1],sigs.shape[2]),sigs], dim=0)

In [5]:
sigs.shape

torch.Size([11648, 6000, 101])

In [10]:
sigs = sigs.reshape(-1,16,6000,sigs.shape[-1])

In [1]:
import SharedArray as sa

In [7]:
X905 = sa.attach('X_905P_22')

In [8]:
X905.shape

(971, 16, 6000, 994)

In [11]:
Xsr1 = np.concatenate([X905[:sigs.shape[0],...],sigs],axis=3)

In [ ]:
try:
    saxsr = sa.create('Xsr1', Xsr1.shape, dtype=np.float32)
    saxsr[:] = Xsr1
except:
    sa.delete('Xsr1')
    saxsr = sa.create('Xsr1', Xsr1.shape, dtype=np.float32)
    saxsr[:] = Xsr1